In [1]:
import semiolog as slg

In [2]:
semiotic = slg.Cenematic("hf_tokenizers")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using custom data configuration corpus-f47d7210eb8e3cef
Reusing dataset text (/Users/Gianni/.cache/huggingface/datasets/text/corpus-f47d7210eb8e3cef/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)
100%|██████████| 3/3 [00:00<00:00, 1319.24it/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForM

In [3]:
sent = semiotic.corpus.train["text"][45]

In [4]:
input = semiotic.syntagmatic.bert_tokenizer(sent)
input

{'input_ids': [295, 329, 10, 1045, 233, 2344, 8, 1, 4961, 6482, 19, 57, 1, 8, 3369, 713, 2477, 2848], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [5]:
semiotic.syntagmatic.bert_tokenizer.decode(
    input["input_ids"]
    )

'thereis still the development time task of [UNK] thecorrect root for a [UNK] of approximately 7 0000 words'

In [6]:
input = semiotic.paradigmatic.bert_tokenizer(sent, verbose = False)
input

2021-12-16 01:12:55.022973: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-16 01:12:55.023103: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


{'input_ids': <tf.Tensor: shape=(1, 18), dtype=int32, numpy=
array([[ 295,  329,   10, 1045,  233, 2344,    8,    1, 4961, 6482,   19,
          57,    1,    8, 3369,  713, 2477, 2848]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 18), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 18), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
      dtype=int32)>}

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

